ΕΡΩΤΗΜΑ 1

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import csv
import numpy as np

Αποθήκευση των csv αρχείων σε dataframe

In [2]:
books = pd.read_csv('BX-Books.csv')
users = pd.read_csv("BX-Users.csv")
ratings = pd.read_csv("BX-Book-Ratings.csv")

In [3]:
ratings = pd.merge(ratings, users, on='uid', how='outer', indicator='Exist')
ratings = ratings[(ratings['Exist']=='both')]
ratings = ratings[['uid', 'isbn', 'rating']]

In [4]:
ratings = pd.merge(ratings, books, on='isbn', how='outer', indicator='Exist')
ratings = ratings[(ratings['Exist']=='both')]
ratings = ratings[['uid', 'isbn', 'rating']]
ratings['uid'] = pd.to_numeric(ratings['uid'], errors='coerce').fillna(0).astype(int)

In [5]:
ratings

,uid,isbn,rating
0,2,0195153448,0.0
1,123629,0002005018,9.0
2,169187,0374157065,0.0
3,169187,155704256X,0.0
4,198013,0399135782,7.0
...,...,...,...
42703,278260,0689821085,5.0
42704,278330,0898861411,5.0
42705,278621,1550390961,8.0
42706,278713,0670528951,8.0


In [6]:
books

,isbn,book_title,book_author,year_of_publication,publisher,summary,category
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Provides an introduction to classical myths pl...,['social science']
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",['actresses']
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o...",['1940-1949']
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an o...",['medical']
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,A look at the incredibly well-preserved ancien...,['design']
...,...,...,...,...,...,...,...
134687,0312266448,The Military Quotation Book : Revised and Expa...,James Charlton,2002,Thomas Dunne Books,"Contains more than 1,200 quotations about war,...",['reference']
134688,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987,Pocket Books,A tongue-in-cheek survival guide for single pe...,['humor']
134689,0743203763,As Hogan Said . . . : The 389 Best Things Anyo...,Randy Voorhees,2000,Simon & Schuster,Golf lovers will revel in this collection of t...,['humor']
134690,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001,Broadway Books,A daring twist on the travel-adventure genre t...,['nature']


Αποθήκευση των βιβλίων στην elasticsearch

In [7]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600, hosts=ENDPOINT)
es.ping()

True

In [8]:
if (not es.indices.exists(index="books")):
    with open('BX-Books.csv', encoding="utf8") as f:
        reader = csv.DictReader(f)
        helpers.bulk(es, reader, index='books')
else:
    print("This index already exists")

This index already exists


Δίνεται το αναγνωριστικό του χρήστη και το αλφαριθμητικό και επιστρέφεται το 10% των βιβλίων που ταιριάζουν καλύτερα με το αλφαριθμητικό σε φρίνουσα σειρά συνυπολογίζοντας την μετρική ομοιότητας της elasticsearch, καθώς και την βαθμολογία που έχει βάλει ο χρήστης

In [9]:
while True:
    user = int(input("Write id of user: "))
    if (users['uid'].isin([user]).any()):
        break
    else:
        print(user, "is not a valid id. Please input a valid user id!")
user_query = input("Write search term: ")
print("uid =", user, end=', ')
print("query =", user_query)

uid = 123629, query = Canada


In [10]:
good_new_ratings = ratings[(ratings["uid"] == user) & (ratings['rating'] > 0)].sort_values(by='rating', ascending=False)
good_rated_books = good_new_ratings['isbn'].tolist()
print(good_rated_books)

['0002005018']


In [11]:
books_ex1 = es.search(index="books", 
  body = {
    "query": {
        "bool": {
            "should": [
                {
                    "multi_match": {
                        "query": user_query,
                        "fields": [
                            "summary^2",
                            "title"
                        ]
                    }
                },
                {
                    "bool": {
                        "should": [
                            {
                                "match": {
                                    "isbn": {
                                        "query": book,
                                        "minimum_should_match": "1",
                                        "boost": 4.0
                                    }
                                }
                            } for book in good_rated_books
                        ]
                    }
                }
            ]
        }
    },
    "size": 10000
  }
)
result = books_ex1["hits"]
total_hits = result['total']['value']

In [12]:
result = books_ex1["hits"]["hits"][:int(0.1*total_hits)]
li = []
for i in result:
  res = i["_source"]
  li.append(res['isbn'])
print(li)

['0002005018', '0968067824', '0717282201', '052164626X', '0792271521', '0870444131', '1551580004', '0771040997', '0195417836', '0770418686', '0060183284', '0919891276', '082630706X', '0618197338', '1551053020', '0395442567', '0395544092', '0618131736', '0395711797', '0395681022', '0395636272', '0395383986', '0395636280', '0618117490', '0395926890', '0864923155', '1553373405', '039593916X', '0395939186']


ΕΡΩΤΗΜΑ 2

In [13]:
from sklearn.cluster import KMeans
from collections import Counter

Θα αποθηκεύσουμε τους χρήστες σε ένα νέο dataframe. Στη συνέχεια θα το τροποποιήσουμε, ώστε να εφαρμόσουμε clustering

In [14]:
new_users = pd.read_csv("BX-Users.csv")

In [15]:
new_users['age'].fillna(users['age'].mean(), inplace=True)

In [16]:
new_users['location'] = new_users['location'].str.split(',')
location_info = pd.DataFrame(new_users["location"].tolist()).fillna('').add_prefix('country_')
new_users = pd.concat([new_users, location_info], axis=1)

new_users.drop(['location', 'country_0', 'country_1', 'country_3', 'country_4', 'country_5', 'country_6', 'country_7', 
                'country_8'], axis=1, inplace=True)
new_users.rename(columns={"country_2": "country"}, inplace=True)

In [17]:
for i, place in enumerate(new_users['country']):
    place = place.strip()
    if (place == '' or place == 'n/a'):
        new_users.at[i, 'country'] = 'usa'
    elif (place.isalpha() == False):
        new_users.at[i, 'country'] = 'usa'
    else:
        new_users.at[i, 'country'] = place

new_users

,uid,age,country
0,1,34.565775,usa
1,2,18.000000,usa
2,3,34.565775,russia
3,4,17.000000,portugal
4,5,34.565775,usa
...,...,...,...
206442,278853,17.000000,usa
206443,278855,50.000000,usa
206444,278856,34.565775,canada
206445,278857,34.565775,usa


In [18]:
users_enc = pd.get_dummies(new_users,columns=["country"])
users_enc

,uid,age,country_a,country_aaa,country_aberdeenshire,country_acoruña,country_adsgfdr,country_afghanistan,country_africa,country_ahrensburg,...,country_yanhill,country_yemen,country_yorkshire,country_ysa,country_yugoslavia,country_yunling,country_z,country_zambia,country_zhengjiang,country_zimbabwe
0,1,34.565775,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,18.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,34.565775,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,17.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,34.565775,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206442,278853,17.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206443,278855,50.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206444,278856,34.565775,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206445,278857,34.565775,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Δημιουργούμε 3 συστάδες για τους χρήστες ανάλογα με την ηλικία και την χώρα κατοικίας τους και τις αποθηκεύουμε στα αντίστοιχα dataframes

In [19]:
kmeans = KMeans(3)
label = kmeans.fit_predict(users_enc.drop(columns='uid'))
cluster_count=Counter(label)
cluster_count

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Counter({0: 129053, 1: 49658, 2: 27736})

In [20]:
users1=new_users[kmeans.labels_==0]
users2=new_users[kmeans.labels_==1]
users3=new_users[kmeans.labels_==2]

Για κάθε συστάδα χρηστών φτίαχνουμε ένα dataframe στο οποίο βρίσκονται όλες οι αξιολογήσεις τους

In [21]:
ratings1=pd.merge(ratings, users1, on='uid', how='outer', indicator='Exist')
ratings1=ratings1[(ratings1['Exist']=='both')]
ratings1.drop(['age', 'country', 'Exist'],axis=1, inplace=True)

In [22]:
ratings2=pd.merge(ratings, users2, on='uid', how='outer', indicator='Exist')
ratings2=ratings2[(ratings2['Exist']=='both')]
ratings2.drop(['age', 'country', 'Exist'],axis=1, inplace=True)

In [23]:
ratings3=pd.merge(ratings, users3, on='uid', how='outer', indicator='Exist')
ratings3=ratings3[(ratings3['Exist']=='both')]
ratings3.drop(['age', 'country', 'Exist'],axis=1, inplace=True)

Στη συνέχεια συμπληρώνουμε για κάθε βιβλίο που έχει αξιολογηθεί την μέση βαθμολογία που έχουν βάλει σε αυτό οι χρήστες της αντίστοιχης συστάδας

In [24]:
ratings1 = ratings1.groupby(['isbn'])['rating'].mean().reset_index(name='average')
books1 = pd.merge(ratings1, books, on='isbn', how='outer')
ratings1 = ratings1[(ratings1['average'] > 0)].sort_values(by='average', ascending=False)
ratings1 = ratings1['isbn'].tolist()

In [25]:
ratings2 = ratings2.groupby(['isbn'])['rating'].mean().reset_index(name='average')
books2 = pd.merge(ratings2, books, on='isbn', how='outer')
ratings2 = ratings2[(ratings2['average'] > 0)].sort_values(by='average', ascending=False)
ratings2 = ratings2['isbn'].tolist()

In [26]:
ratings3 = ratings3.groupby(['isbn'])['rating'].mean().reset_index(name='average')
books3 = pd.merge(ratings3, books, on='isbn', how='outer')
ratings3 = ratings3[(ratings3['average'] > 0)].sort_values(by='average', ascending=False)
ratings3 = ratings3['isbn'].tolist()

In [27]:
user_cluster = 0
good_rated_books2 = good_rated_books.copy()
if (users1['uid'].isin([user]).any()):
    user_cluster = 1
    good_rated_books2.extend(ratings1)
elif (users2['uid'].isin([user]).any()):
    user_cluster = 2
    good_rated_books2.extend(ratings2)
elif (users3['uid'].isin([user]).any()):
    user_cluster = 3
    good_rated_books2.extend(ratings3)
good_rated_books2 = good_rated_books2[:1024]

In [28]:
books_ex2 = es.search(index="books", 
  body = {
    "query": {
        "bool": {
            "should": [
                {
                    "multi_match": {
                        "query": user_query,
                        "fields": [
                            "summary^2",
                            "title"
                        ]
                    }
                },
                {
                    "bool": {
                        "should": [
                            {
                                "match": {
                                    "isbn": {
                                        "query": book,
                                        "minimum_should_match": "1",
                                        "boost": 4.0
                                    }
                                }
                            } for book in good_rated_books2
                        ]
                    }
                }
            ]
        }
    },
    "size": 10000
  }
)
result = books_ex2["hits"]
total_hits = result['total']['value']

In [29]:
result = books_ex2["hits"]["hits"][:int(0.1*total_hits)]
li = []
for i in result:
  res = i["_source"]
  li.append(res['isbn'])
print(li)

['0871238829', '0143312146', '0002005018', '0743223527', '1583485473', '0595206263', '0968288308', '1879384493', '0966986105', '0840734530', '059043893X', '0345391691', '0060973897', '1573225126', '0679310002', '0440225078', '193072229X', '0609605925', '1563411148', '0945397410', '0140274154', '0345409469', '0140304770', '0345307674', '0394404289', '0380012774', '0743444051', '0671619055', '0743412273', '0307001385', '0679439242', '0816614024', '1587991039', '0440512158', '0765342294', '0316919896', '0373871864', '0515107662', '0140286780', '0380810336', '0451456718', '0812533550', '0836220889', '0060923717', '0345421825', '0590629808', '0590956159', '0671517643', '0312311354', '034536662X', '0786404019', '0446603775', '0553277537', '0399147101', '0500280673', '0440228204', '0026890380', '0140309578', '0312180624', '1561794708', '1566840287', '0060504080', '1573226106', '0517576988', '0451201515', '1583940634', '0449212858', '0140344438', '0142000345', '0376026138', '0440226848', '0679

ΕΡΩΤΗΜΑ 3

In [30]:
from gensim.models import Word2Vec
import gensim
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model , Sequential
import tensorflow as tf
from tensorflow import keras

Αρχικά κάνουμε preprocessing στις περιλήψεις των βιβλίων

In [31]:
new_books= books[['summary']]
summaries = new_books.summary.apply(gensim.utils.simple_preprocess)

Στη συνέχεια μέσω της gensim μετατρέπουμε κάθε λέξη σε διάνυσμα και προσθέτουμε τα διανύσματα των λέξεων για κάθε περίληψη

In [32]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

model.build_vocab(summaries, progress_per = 1000)
model.train(summaries, total_examples=model.corpus_count, epochs=model.epochs)

(13862105, 17864510)

In [33]:
li = []
i = 0
j = 0
arr = np.empty([100, ], dtype = np.float32)
for review in summaries:
    for word in review:
        try:
            arr += model.wv.get_vector(word)
            j += 1
        except Exception as e:
            arr += np.zeros([100, ])
            i += 1
    li.append(arr)
    arr = np.empty([100, ], dtype = np.float32)

C:\Users\dniko\AppData\Local\Temp\ipykernel_9956\1344823315.py:8: RuntimeWarning: invalid value encountered in add
  arr += model.wv.get_vector(word)


Για κάθε μια από τις συστάδες χωρίζουμε τα βιβλία με αξιολογήσεις, από αυτά που δεν έχουν αξιολογηθεί

In [34]:
new_books1 = books1
new_books1['summary']=li
new_books2 = books2
new_books2['summary']=li
new_books3 = books3
new_books3['summary']=li

In [35]:
rated_books1 = new_books1[new_books1['average'].notna()]
unrated_books1 = new_books1[new_books1['average'].isna()]
unrated_books1.reset_index(drop=True, inplace=True)

In [36]:
rated_books2 = new_books2[new_books2['average'].notna()]
unrated_books2 = new_books2[new_books2['average'].isna()]
unrated_books2.reset_index(drop=True, inplace=True)

In [37]:
rated_books3 = new_books3[new_books3['average'].notna()]
unrated_books3 = new_books3[new_books3['average'].isna()]
unrated_books3.reset_index(drop=True, inplace=True)

Κανονικοποιούμε τα δεδομένα και τα μετατρέπουμε σε κατάλληλη μορφή για να εισέλθουν στο νευρωνικό δίκτυο

In [38]:
X1 = rated_books1[['summary']]
X1_pred = unrated_books1[['summary']]
y1 = rated_books1['average']

X1 = np.array(X1.summary.tolist()).reshape(-1, len(X1.summary[0]))
X1_pred = np.array(X1_pred.summary.tolist()).reshape(-1, len(X1_pred.summary[0]))
y1 = np.array(y1)


X1_pred[np.isnan(X1_pred)] = 0.0001
X1[np.isnan(X1)] = 0.0001

In [39]:
m = -1000000000000000
l = -1000000000000000
for i in X1:
    if i.max() > m:
        m = i.max()
for i in y1:
    if i.max() > l:
        l = i.max()

X1 = X1 / m

m = -1000000000000000
l = -1000000000000000
for i in X1_pred:
    if i.max() > m:
        m = i.max()
for i in y1:
    if i.max() > l:
        l = i.max()

X1_pred = X1_pred / m

In [40]:
X2 = rated_books2[['summary']]
X2_pred = unrated_books2[['summary']]
y2 = rated_books2['average']

X2 = np.array(X2.summary.tolist()).reshape(-1, len(X2.summary[0]))
X2_pred = np.array(X2_pred.summary.tolist()).reshape(-1, len(X2_pred.summary[0]))
y2 = np.array(y2)


X2_pred[np.isnan(X2_pred)] = 0.0001
X2[np.isnan(X2)] = 0.0001

In [41]:
m = -1000000000000000
l = -1000000000000000
for i in X2:
    if i.max() > m:
        m = i.max()
for i in y2:
    if i.max() > l:
        l = i.max()

X2 = X2 / m

m = -1000000000000000
l = -1000000000000000
for i in X2_pred:
    if i.max() > m:
        m = i.max()
for i in y2:
    if i.max() > l:
        l = i.max()

X2_pred = X2_pred / m

In [42]:
X3 = rated_books3[['summary']]
X3_pred = unrated_books3[['summary']]
y3 = rated_books3['average']

X3 = np.array(X3.summary.tolist()).reshape(-1, len(X3.summary[0]))
X3_pred = np.array(X3_pred.summary.tolist()).reshape(-1, len(X3_pred.summary[0]))
y3 = np.array(y3)


X3_pred[np.isnan(X3_pred)] = 0.0001
X3[np.isnan(X3)] = 0.0001

In [43]:
m = -1000000000000000
l = -1000000000000000
for i in X3:
    if i.max() > m:
        m = i.max()
for i in y3:
    if i.max() > l:
        l = i.max()

X3 = X3 / m

m = -1000000000000000
l = -1000000000000000
for i in X3_pred:
    if i.max() > m:
        m = i.max()
for i in y3:
    if i.max() > l:
        l = i.max()

X3_pred = X3_pred / m

In [44]:
y1 = np.rint(y1).astype(np.int64)
y2 = np.rint(y2).astype(np.int64)
y3 = np.rint(y3).astype(np.int64)

Εκπαιδεύουμε το νευρωνικό μας δίκτυο ώστε να δέχεται σαν είσοδο τα διανύσματα των περιλήψεων και να προβλέπει την αξιολόγηση των βιβλίων με βάση τις ήδη υπάρχουσες αξιολογήσεις κάθε συστάδας

In [45]:

model1 = keras.Sequential([
    keras.layers.Dense(100, input_shape=(100, ), activation='relu'),
    keras.layers.Dense(11, activation='softmax')
])

model1.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
)

model1.fit(X1, y1, epochs=3)

Epoch 1/3
467/467 [==============================] - 3s 3ms/step - loss: 2.1280 - accuracy: 0.3152
Epoch 2/3
467/467 [==============================] - 1s 3ms/step - loss: 2.0828 - accuracy: 0.3155
Epoch 3/3
467/467 [==============================] - 1s 3ms/step - loss: 2.0798 - accuracy: 0.3155


In [46]:
new_average1=model1.predict(X1_pred)
predicted_average1 = [np.argmax(i) for i in new_average1]
unrated_books1['average'] = predicted_average1

3744/3744 [==============================] - 8s 2ms/step


C:\Users\dniko\AppData\Local\Temp\ipykernel_9956\3444542414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unrated_books1['average'] = predicted_average1


In [47]:
model2 = keras.Sequential([
    keras.layers.Dense(100, input_shape=(100, ), activation='relu'),
    keras.layers.Dense(11, activation='softmax')
])

model2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
)

model2.fit(X2, y2, epochs=3)

Epoch 1/3
172/172 [==============================] - 1s 2ms/step - loss: 2.1484 - accuracy: 0.3482
Epoch 2/3
172/172 [==============================] - 0s 3ms/step - loss: 1.9885 - accuracy: 0.3526
Epoch 3/3
172/172 [==============================] - 0s 2ms/step - loss: 1.9808 - accuracy: 0.3526


In [48]:
new_average2=model2.predict(X2_pred)
predicted_average2 = [np.argmax(i) for i in new_average2]
unrated_books2['average'] = predicted_average2

4038/4038 [==============================] - 7s 2ms/step


C:\Users\dniko\AppData\Local\Temp\ipykernel_9956\3729225543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unrated_books2['average'] = predicted_average2


In [49]:
model3 = keras.Sequential([
    keras.layers.Dense(100, input_shape=(100, ), activation='relu'),
    keras.layers.Dense(11, activation='softmax')
])

model3.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
)

model3.fit(X3, y3, epochs=3)

Epoch 1/3
137/137 [==============================] - 1s 2ms/step - loss: 2.2022 - accuracy: 0.2969
Epoch 2/3
137/137 [==============================] - 0s 3ms/step - loss: 2.0282 - accuracy: 0.3145
Epoch 3/3
137/137 [==============================] - 0s 2ms/step - loss: 2.0178 - accuracy: 0.3147


In [50]:
new_average3=model3.predict(X3_pred)
predicted_average3 = [np.argmax(i) for i in new_average3]
unrated_books3['average'] = predicted_average3

4073/4073 [==============================] - 6s 2ms/step


C:\Users\dniko\AppData\Local\Temp\ipykernel_9956\3209853018.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unrated_books3['average'] = predicted_average3


Προσθέτουμε τις προβλεπόμενες βαθμολογίες κάθε βιβλίου 

In [51]:
final_books1 = pd.concat([rated_books1, unrated_books1], axis=0)
final_books1.drop('summary', axis=1, inplace=True)
b1=books1[['isbn', 'summary']]
final_books1= pd.merge(b1, final_books1, on='isbn', how='outer')
final_books1 = final_books1[(final_books1['average'] > 0)].sort_values(by='average', ascending=False)
final_books1 = final_books1['isbn'].tolist()

In [52]:
final_books2 = pd.concat([rated_books2, unrated_books2], axis=0)
final_books2.drop('summary', axis=1, inplace=True)
b2=books2[['isbn', 'summary']]
final_books2= pd.merge(b2, final_books2, on='isbn', how='outer')
final_books2 = final_books2[(final_books2['average'] > 0)].sort_values(by='average', ascending=False)
final_books2 = final_books2['isbn'].tolist()

In [53]:
final_books3 = pd.concat([rated_books3, unrated_books3], axis=0)
final_books3.drop('summary', axis=1, inplace=True)
b3=books3[['isbn', 'summary']]
final_books3= pd.merge(b3, final_books3, on='isbn', how='outer')
final_books3 = final_books3[(final_books3['average'] > 0)].sort_values(by='average', ascending=False)
final_books3 = final_books3['isbn'].tolist()

In [54]:
good_rated_books3 = good_rated_books.copy()
if (user_cluster == 1):
    good_rated_books3.extend(final_books1)
elif (user_cluster == 2):
    good_rated_books3.extend(final_books2)
elif (user_cluster == 3):
    good_rated_books3.extend(final_books3)
good_rated_books3 = good_rated_books3[:1024]

In [56]:
books_ex3 = es.search(index="books", 
  body = {
    "query": {
        "bool": {
            "should": [
                {
                    "multi_match": {
                        "query": user_query,
                        "fields": [
                            "summary^2",
                            "title"
                        ]
                    }
                },
                {
                    "bool": {
                        "should": [
                            {
                                "match": {
                                    "isbn": {
                                        "query": book,
                                        "minimum_should_match": "1",
                                        "boost": 4.0
                                    }
                                }
                            } for book in good_rated_books3
                        ]
                    }
                }
            ]
        }
    },
    "size": 10000
  }
)
result = books_ex3["hits"]
total_hits = result['total']['value']

In [57]:
result = books_ex3["hits"]["hits"][:int(0.1*total_hits)]
li = []
for i in result:
  res = i["_source"]
  li.append(res['isbn'])
print(li)

['0871238829', '0143312146', '0002005018', '0743223527', '1583485473', '0595206263', '0968288308', '1879384493', '0964778319', '0966986105', '0840734530', '059043893X', '0345391691', '0060973897', '1573225126', '0679310002', '0440225078', '193072229X', '0609605925', '1563411148', '0945397410', '0140274154', '0345409469', '0140304770', '0345307674', '0394404289', '0380012774', '0743444051', '0671619055', '0743412273', '0307001385', '0679439242', '0816614024', '1587991039', '0440512158', '0765342294', '0316919896', '0373871864', '0440168724', '0515107662', '0140286780', '0380810336', '0451456718', '0812533550', '0836220889', '0060923717', '0345421825', '0590629808', '0590956159', '0671517643', '0312311354', '034536662X', '0786404019', '0446603775', '0553277537', '0399147101', '0500280673', '0440228204', '0026890380', '0140309578', '0312180624', '1561794708', '0060504080', '1573226106', '0440183057', '0517576988', '0451201515', '1583940634', '0449212858', '0140344438', '0142000345', '0376